In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\yasba\\OneDrive\\Documents\\Projects\\Audio_Classification_UrbanSound8K'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    checkpoint_model_filepath: Path
    train_data_dir: Path
    test_data_dir: Path
    base_model_path: Path
    params_epochs: int
    params_batch_size: int

In [5]:
from Audio_Classification.constants import *
from Audio_Classification.utils.common import read_yaml, create_directories
from Audio_Classification import logger

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


    def get_training_config(self) -> ModelTrainingConfig:
        config = self.config.training
        params = self.params

        create_directories([config.root_dir])

        trainig_data_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            train_data_dir = config.train_data_dir,
            test_data_dir = config.test_data_dir,
            trained_model_path = config.trained_model_path,
            checkpoint_model_filepath = config.checkpoint_model_filepath,
            base_model_path = config.base_model_path,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
        )

        return trainig_data_config

In [7]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
import pandas as pd 
import numpy as np
import ast
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [8]:
class Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config


    def get_train_test_data(self):
        self.train_df = pd.read_csv(self.config.train_data_dir)
        self.test_df = pd.read_csv(self.config.test_data_dir)

        X_train = self.train_df["mfccs"].apply(lambda x: np.fromstring(x.replace('\n', ' ').strip('[]'), sep=' '))
        self.X_train = np.vstack(X_train)
        self.y_train = np.array(self.train_df["class"].tolist())

        X_test = self.test_df["mfccs"].apply(lambda x: np.fromstring(x.replace('\n', ' ').strip('[]'), sep=' '))
        self.X_test = np.vstack(X_test)
        self.y_test = np.array(self.test_df["class"].tolist())

        
    def get_base_model(self):
        self.model = Sequential()

        ## First Layer
        self.model.add(Dense(100,input_shape=(40,)))
        self.model.add(Activation("relu"))
        self.model.add(Dropout(0.5))

        ## Second Layer
        self.model.add(Dense(200))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        ## Third Layer
        self.model.add(Dense(100))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        ## Final Layer
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        ## Compile the self.model
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.model.summary()

    def train(self):
        checkpointer = ModelCheckpoint(filepath=self.config.checkpoint_model_filepath, verbose=1, save_best_only=True)

        start = datetime.now()

        self.model.fit(self.X_train, self.y_train,
                       batch_size=self.config.params_batch_size, 
                       epochs=self.config.params_epochs, 
                       validation_data=(self.X_test, self.y_test), 
                       callbacks=[checkpointer]
                    )

        duration = datetime.now() - start
        print("Training completed in time: ",duration)

    
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_train_test_data()
    training.get_base_model()
    training.train()
except Exception as e:
    raise e

[2024-12-24 02:29:29,641: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-24 02:29:29,645: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-24 02:29:29,647: INFO: common: created directory at: artifacts/training]
(6985, 40)
(1747, 40)
(6985,)
(1747,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation